# F-тест: значимость регрессии

Тестируется гипотеза $H_0:\beta_1=\cdots=\beta_k=0$

Тестовая статистика $F$ (расчитывается автоматически)

Критическое значение $F_{cr}=F_{df1=k, df2=n-k-1}(\alpha)$

Гипотеза отвергается, если $F>F_{cr}$ или $P<\alpha$

# LPM-модель: F-тест на значимость регрессии

In [11]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col, summary_params # вывод результатов тестирования
from scipy.stats import f # F-распределение

### approve equation #1

Для датасета `loanapp` рассморим регрессию **approve на unem, male, yjob, self**

Спецификация: 
$$
  approve=\beta_0+\beta_1unem+\beta_2male+\beta_3yjob+\beta_4self+u
$$

Альтернативная спецификация:
$$
  P(approve=1)=\beta_0+\beta_1unem+\beta_2male+\beta_3yjob+\beta_4self
$$

In [12]:
# загрузим данные
loanapp_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/econometrica/main/econometrica2/data-csv/loanapp.csv', na_values=(' ', '', '  '))

In [13]:
#зададим спецификацию модели через формулу
mod_lpm = smf.ols(formula='approve~unem+male+yjob+self', data=loanapp_df)

In [14]:
# подгонка модели с поправкой на гетероскедастичность
res_lpm_hc = mod_lpm.fit(cov_type='HC3')
print(res_lpm_hc.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                approve   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.002
No. Observations:                1974   F-statistic:                     1.677
Covariance Type:                  HC3   Prob (F-statistic):              0.153
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8914      0.023     39.586      0.000       0.847       0.936
unem          -0.0074      0.004     -1.838      0.066      -0.015       0.000
male           0.0209      0.020      1.056      0.291      -0.018       0.060
yjob           0.0013      0.006      0.210      0.834      -0.011       0.013
self          -0.0298      0.025     -1.205      0.228      -0.078       0.019

Notes:
[1] Standard Errors are heteroscedasticity r

### Тестируем значимость регрессии, т.е. гипотезу $$H_0:\beta_{unem}=\beta_{male}=\beta_{yjob}=\beta_{self}=0.$$

### Результаты робастного F-теста

In [15]:
# тестовая статистика и P-значение
np.round(res_lpm_hc.fvalue, 3), np.round(res_lpm_hc.f_pvalue, 3)

(1.677, 0.153)

### Результаты неробастного F-теста

In [16]:
# подгонка модели
res_lpm_ols = mod_lpm.fit(cov_type='nonrobust')
# тестовая статистика и P-значение
res_lpm_ols.fvalue.round(3), res_lpm_ols.f_pvalue.round(3)

(2.029, 0.088)

In [17]:
# зададим уровень значимости
alpha = 0.1

### 10\%-критическое значение F-распределения

In [18]:
f.ppf(q=1-0.1, dfn=res_lpm_hc.df_model, dfd=res_lpm_hc.df_resid).round(3)

1.948

In [19]:
# можно использовать переменную alpha
f.ppf(q=1-alpha, dfn=res_lpm_hc.df_model, dfd=res_lpm_hc.df_resid).round(3)

1.948

## **Вывод:** регрессия незначима!

### Альтернативный способ: явно специфицируем тестируемую гипотезу

In [20]:
# тестовая статистика, P-значение и степени свободы
print(res_lpm_hc.f_test('unem=0, male=0, yjob=0, self=0'))

<F test: F=1.6774796752587853, p=0.15250462834382744, df_denom=1.97e+03, df_num=4>
